In [ ]:
'''Imports '''

import pandas as pd
import numpy as np
from datetime import datetime

import nfl_data_py as nfl

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio

from resources.plotly_theme import nfl_template

pio.templates['nfl_template'] = nfl_template

# Quarterbacks

In [ ]:
''' QB Accuracy '''

YEAR = 2025

# Logos
team_data = nfl.import_team_desc() 

# PFR
passers = nfl.import_seasonal_pfr(s_type='pass', years=[YEAR])
passers = passers.sort_values(by='team', ascending=True).drop_duplicates(subset='player').reset_index(drop=True)

# NGS
ngs = nfl.import_ngs_data(stat_type='passing', years=[YEAR])
ngs = ngs.loc[ngs['week'] == 0, :].reset_index(drop=True)
ngs.loc[ngs['player_display_name'] == 'Michael Penix Jr.', 'player_display_name'] = 'Michael Penix'
ngs.loc[ngs['player_display_name'] == 'Gardner Minshew', 'player_display_name'] = 'Gardner Minshew II'
ngs.loc[ngs['player_display_name'] == 'Cameron Ward', 'player_display_name'] = 'Cam Ward'

# Merge
passers = passers.merge(ngs[['player_display_name', 'completions', 'completion_percentage']], left_on='player', right_on='player_display_name', how='left')
passers['adj_pass_attempts'] = passers['pass_attempts'] - passers['throwaways'] - passers['batted_balls'] - passers['drops'] - passers['spikes']
passers['adj_completion_percentage'] = passers['completions'] / passers['adj_pass_attempts']

passers = passers.merge(team_data[['team_abbr', 'team_color', 'team_logo_espn']], left_on='team', right_on='team_abbr', how='left').drop(columns=['team_abbr'])

passers = passers.loc[passers['pass_attempts'] >= 144, :]

passers['text'] = passers['completions'].astype(int).astype(str) + ' / ' + passers['adj_pass_attempts'].astype(int).astype(str) + ' (' + (passers['adj_completion_percentage'] * 100).round(1).astype(str) + '%)'
passers['text2'] = passers['bad_throws'].astype(int).astype(str) + ' (' + passers['bad_throw_pct'].round(1).astype(str) + '%)'

print(passers.shape)
print(passers.head().to_string())

# Adj Completion Pct
passers = passers.sort_values(by='adj_completion_percentage', ascending=False)
adj_comp_y = passers['player'].to_numpy()
adj_comp_logos = passers['team_logo_espn'].to_numpy()
adj_comp_colors = passers['team_color'].to_numpy()
adj_comp_pct = px.bar(
    x=passers['adj_completion_percentage'].to_numpy(),
    y=adj_comp_y,
    orientation='h',
    text=passers['text'].to_numpy()
)
adj_comp_pct.add_vline(x=passers['adj_completion_percentage'].mean(), line_dash='longdash', line_color='red', line_width=1)
adj_comp_pct.update_xaxes(
    tickformat='.1%',
    title='Adj Completion %'
)
adj_comp_pct.update_yaxes(
    autorange='reversed',
    title=None
)
adj_comp_pct.update_layout(
    title=f'<b>{YEAR} Adjusted Completion %</b><br><sup>Removing throwaways, drops, and spikes; min 144 attempts</sup>',
    margin=dict(l=150, r=25, b=40, t=75, pad=5),
    height=900,
    width=700
)
# adj_comp_pct.show()

# Bad throw pct
passers = passers.sort_values(by='bad_throw_pct', ascending=False)
passers['bad_throw_pct'] = passers['bad_throw_pct'] / 100
bad_throw_y = passers['player'].to_numpy()
bad_throw_logos = passers['team_logo_espn'].to_numpy()
bad_throw_colors = passers['team_color'].to_numpy()
bad_throw_pct = px.bar(
    x=passers['bad_throw_pct'].to_numpy(),
    y=bad_throw_y,
    orientation='h',
    text=passers['text2'].to_numpy()
)
bad_throw_pct.add_vline(x=passers['bad_throw_pct'].mean(), line_dash='longdash', line_color='red', line_width=1)
bad_throw_pct.update_xaxes(
    tickformat='.0%',
    title='Bad Throw %'
)
bad_throw_pct.update_yaxes(
    autorange='reversed',
    title=None
)
bad_throw_pct.update_layout(
    title=f'<b>{YEAR} Bad Throw %</b><br><sup>min 144 attempts</sup>',
    margin=dict(l=150, r=25, b=40, t=75, pad=5),
    height=900,
    width=700
)
# bad_throw_pct.show()


## FIGURE ##

H_SPACING = 0.3/2
titles = ['<b>Adjusted Completion %</b><br><sup>Removing throwaways, drops, batted balls, and spikes</sup>', '<b>Bad Throw %</b>']
fig = make_subplots(rows=1, cols=2, horizontal_spacing=H_SPACING, subplot_titles=titles,
                    print_grid=True)

for trace in adj_comp_pct.data:
    fig.add_trace(trace, row=1, col=1)

for trace in bad_throw_pct.data:
    fig.add_trace(trace, row=1, col=2)

# Plot titles
fig.for_each_annotation(lambda a: a.update(y=1.01, yanchor='bottom', font=dict(size=14)) if a.text in titles else())

# Avg lines
fig.add_vline(x=passers['adj_completion_percentage'].mean(), line_dash='longdash', line_color='red', line_width=1, row=1, col=1)
fig.add_vline(x=passers['bad_throw_pct'].mean(), line_dash='longdash', line_color='red', line_width=1, row=1, col=2)

# Logos
for i in range(len(adj_comp_logos)):
    if type(adj_comp_logos[i]) != str: continue
    fig.add_layout_image(
        source=adj_comp_logos[i],
        xref='paper', yref='y',
        xanchor='center', yanchor='middle',
        x=-.125, y=adj_comp_y[i],
        sizex=1, sizey=1,
        layer='above'
    )
for i in range(len(bad_throw_logos)):
    if type(bad_throw_logos[i]) != str: continue
    fig.add_layout_image(
        source=bad_throw_logos[i],
        xref='paper', yref='y2',
        xanchor='center', yanchor='middle',
        x=((1-H_SPACING) / 2) + (H_SPACING*.15), y=bad_throw_y[i],
        sizex=1, sizey=1,
        layer='above'
    )

# Formatting
fig.update_traces(
    marker=dict(color=adj_comp_colors, opacity=0.8, line=dict(color='#323232', width=1)),
    row=1, col=1
)
fig.update_traces(
    marker=dict(color=bad_throw_colors, opacity=0.8, line=dict(color='#323232', width=1)),
    row=1, col=2
)

fig.update_xaxes(
    tickformat='.0%',
    linecolor='#f0f0f0', mirror=True
)
fig.update_yaxes(
    linecolor='#f0f0f0', mirror=True
)
fig.update_yaxes(
    autorange='reversed',
    col=1
)
fig.update_layout(
    template='nfl_template',
    title=dict(
        text=f'<b>{YEAR} Quarterback Accuracy</b>',
        y=0.965, yref='container'
    ),
    margin=dict(l=150, r=25, b=60, t=100, pad=5),
    height=900, width=1200,
)

# Credits
fig.add_annotation(
    text=f'Min. 144 attempts<br>Figure: @clankeranalytic | Data: nfl_data_py | {datetime.today().strftime("%Y-%m-%d")}',
    showarrow=False,
    xref='paper',
    yref='paper',
    y=-0.075, 
    x=1,
    align='right'
)

fig.show()

In [ ]:
fig = px.scatter(
    x=passers['bad_throw_pct'].to_numpy(),
    y=passers['adj_completion_percentage'],
    size=passers['pass_attempts'],
    text=passers['player'].to_numpy()
    # trendline='ols'
)

fig.update_traces(
    marker=dict(color=passers['team_color'].to_numpy(), opacity=0.7, line=dict(color='#323232', width=1))
)
fig.update_layout(
    height=500,
    width=900
)
fig.show()